In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import random
import matplotlib.pyplot as plt
import seaborn as sns
from src.models.utils import cnc_add_y_label_binary

%load_ext autoreload
%autoreload 2

In [2]:
path_data_dir = Path.cwd().parent.parent / "data"
path_processed_dir = Path(Path.cwd().parent.parent / 'data/processed/cnc/cnc_features_comp')
print(path_processed_dir)

/home/tim/Documents/feat-store/data/processed/cnc/cnc_features_comp


In [3]:
df = pd.read_csv(
    path_processed_dir / "cnc_features_54_comp.csv",
)
df.head()

,id,current__time_reversal_asymmetry_statistic__lag_1,current__time_reversal_asymmetry_statistic__lag_2,current__time_reversal_asymmetry_statistic__lag_3,current__c3__lag_1,current__c3__lag_2,current__c3__lag_3,current__cid_ce__normalize_True,current__cid_ce__normalize_False,current__symmetry_looking__r_0.0,...,current__permutation_entropy__dimension_5__tau_1,current__permutation_entropy__dimension_6__tau_1,current__permutation_entropy__dimension_7__tau_1,current__query_similarity_count__query_None__threshold_0.0,"current__matrix_profile__feature_""min""__threshold_0.98","current__matrix_profile__feature_""max""__threshold_0.98","current__matrix_profile__feature_""mean""__threshold_0.98","current__matrix_profile__feature_""median""__threshold_0.98","current__matrix_profile__feature_""25""__threshold_0.98","current__matrix_profile__feature_""75""__threshold_0.98"
0,1540298934_54_1,-8.443769e+04,-2.462711e+05,-4.599122e+05,1.328925e+09,1.327239e+09,1.325897e+09,4.621212,2723.857742,0.0,...,4.515810,5.997846,7.364881,NaN,4.357131,21.892176,6.575366,5.699876,5.326978,6.176852
1,1540300010_54_2,-7.495608e+06,-1.373315e+07,-2.032414e+07,6.490759e+09,6.464760e+09,6.467292e+09,9.892145,8035.008836,0.0,...,4.297358,5.603116,6.612667,NaN,1.919631,12.545193,5.831122,2.903726,2.449469,10.213633
2,1540301801_54_4,-1.730202e+05,-3.898226e+05,-5.975301e+05,4.771343e+09,4.770460e+09,4.768193e+09,5.553800,2675.926008,0.0,...,4.454407,5.834909,6.843201,NaN,1.165741,3.546220,2.478614,2.657725,2.227665,2.867848
3,1540304666_54_4,-3.288852e+03,-9.183639e+04,-2.449278e+05,5.089232e+09,5.087934e+09,5.085348e+09,5.490867,2726.979465,0.0,...,4.428960,5.802252,6.826084,NaN,1.143674,4.186782,2.284911,2.352185,2.067048,2.579063
4,1540305024_54_2,-7.237124e+06,-1.312405e+07,-2.011030e+07,6.459150e+09,6.433939e+09,6.438002e+09,9.822227,7977.009715,0.0,...,4.260208,5.540055,6.539044,NaN,1.529200,9.563519,3.944657,2.225357,1.852027,6.166976


In [4]:
n_feats = df.shape[1] - 1
n_feats

751

In [5]:
# save n_feats as a dataframe
df_n_feats = pd.DataFrame({"n_feats": [n_feats]})
df_n_feats

,n_feats
0,751


In [7]:


def load_cnc_features(
    path_data_dir, path_processed_dir, feat_file_name, label_file_name
):
    """
    This function returns a dataframe with the appropriate meta-label columns and the label column (y).

    Meta-label columns are:
    - case_tool_54: the case number for tool 54
    - unix_date: the unix date at the time of the cut
    - tool_no: the tool number
    - index_no: the index number of the cut

    """
    df = pd.read_csv(
        path_processed_dir / feat_file_name,
    )
    df["unix_date"] = df["id"].apply(lambda x: int(x.split("_")[0]))
    df["tool_no"] = df["id"].apply(lambda x: int(x.split("_")[-2]))
    df["index_no"] = df["id"].apply(lambda x: int(x.split("_")[-1]))

    df_labels = pd.read_csv(path_data_dir / "processed/cnc" / label_file_name)

    df = cnc_add_y_label_binary(df, df_labels, col_list_case=["case_tool_54"])
    df = df.dropna(axis=1, how="all")  # drop any columns that are completely empty
    df = df.dropna(axis=0)  # drop any rows that have NaN values in them
    return df

feat_file_name = "cnc_features_54_comp.csv"
label_file_name = "high_level_labels_MASTER_update2022-08-18_with_case.csv"

df = load_cnc_features(path_data_dir, path_processed_dir, feat_file_name, label_file_name)
print(df.shape)
df.head()

(43679, 756)


,id,current__time_reversal_asymmetry_statistic__lag_1,current__time_reversal_asymmetry_statistic__lag_2,current__time_reversal_asymmetry_statistic__lag_3,current__c3__lag_1,current__c3__lag_2,current__c3__lag_3,current__cid_ce__normalize_True,current__cid_ce__normalize_False,current__symmetry_looking__r_0.0,...,"current__matrix_profile__feature_""max""__threshold_0.98","current__matrix_profile__feature_""mean""__threshold_0.98","current__matrix_profile__feature_""median""__threshold_0.98","current__matrix_profile__feature_""25""__threshold_0.98","current__matrix_profile__feature_""75""__threshold_0.98",unix_date,tool_no,index_no,case_tool_54,y
0,1540298934_54_1,-8.443769e+04,-2.462711e+05,-4.599122e+05,1.328925e+09,1.327239e+09,1.325897e+09,4.621212,2723.857742,0.0,...,21.892176,6.575366,5.699876,5.326978,6.176852,1540298934,54,1,1,0
1,1540298934_54_2,-6.912358e+06,-1.310691e+07,-1.931507e+07,6.473963e+09,6.450330e+09,6.453858e+09,9.929669,8008.548807,0.0,...,7.838717,3.066461,1.857761,1.624711,4.336304,1540298934,54,2,1,0
2,1540298934_54_5,-2.707128e+05,-4.525839e+05,-5.188306e+05,3.376005e+09,3.375504e+09,3.373084e+09,4.389280,2097.344273,0.0,...,3.672813,2.370622,2.496121,1.804606,2.813998,1540298934,54,5,1,0
3,1540298934_54_3,-3.874679e+04,1.883553e+04,9.845639e+04,2.574682e+09,2.572420e+09,2.568122e+09,7.268473,2443.065083,0.0,...,3.613585,1.835852,1.862852,1.627676,2.047316,1540298934,54,3,1,0
4,1540298934_54_9,1.316437e+04,1.437149e+05,3.375137e+05,4.153671e+08,4.110654e+08,4.038862e+08,26.105369,2815.061101,0.0,...,8.261546,5.619089,5.728254,4.895724,5.969812,1540298934,54,9,1,0


In [8]:
# remove any duplicates by unix_date
df = df.drop_duplicates(subset=["unix_date"])
print(df.shape)
df.head()

(5503, 756)


,id,current__time_reversal_asymmetry_statistic__lag_1,current__time_reversal_asymmetry_statistic__lag_2,current__time_reversal_asymmetry_statistic__lag_3,current__c3__lag_1,current__c3__lag_2,current__c3__lag_3,current__cid_ce__normalize_True,current__cid_ce__normalize_False,current__symmetry_looking__r_0.0,...,"current__matrix_profile__feature_""max""__threshold_0.98","current__matrix_profile__feature_""mean""__threshold_0.98","current__matrix_profile__feature_""median""__threshold_0.98","current__matrix_profile__feature_""25""__threshold_0.98","current__matrix_profile__feature_""75""__threshold_0.98",unix_date,tool_no,index_no,case_tool_54,y
0,1540298934_54_1,-8.443769e+04,-2.462711e+05,-4.599122e+05,1.328925e+09,1.327239e+09,1.325897e+09,4.621212,2723.857742,0.0,...,21.892176,6.575366,5.699876,5.326978,6.176852,1540298934,54,1,1,0
10,1540300010_54_2,-7.495608e+06,-1.373315e+07,-2.032414e+07,6.490759e+09,6.464760e+09,6.467292e+09,9.892145,8035.008836,0.0,...,12.545193,5.831122,2.903726,2.449469,10.213633,1540300010,54,2,1,0
20,1540301801_54_4,-1.730202e+05,-3.898226e+05,-5.975301e+05,4.771343e+09,4.770460e+09,4.768193e+09,5.553800,2675.926008,0.0,...,3.546220,2.478614,2.657725,2.227665,2.867848,1540301801,54,4,1,0
30,1540304666_54_4,-3.288852e+03,-9.183639e+04,-2.449278e+05,5.089232e+09,5.087934e+09,5.085348e+09,5.490867,2726.979465,0.0,...,4.186782,2.284911,2.352185,2.067048,2.579063,1540304666,54,4,1,0
40,1540305024_54_2,-7.237124e+06,-1.312405e+07,-2.011030e+07,6.459150e+09,6.433939e+09,6.438002e+09,9.822227,7977.009715,0.0,...,9.563519,3.944657,2.225357,1.852027,6.166976,1540305024,54,2,1,0


In [9]:
n_sub_cuts = df.id.unique().shape[0]
print("n_sub_cuts:", n_sub_cuts)

n_sub_cuts: 5503


In [10]:
# get the percentage of each y label
df_p = df.groupby('y').size() / df.shape[0] * 100
df_p = df_p.reset_index()
df_p.columns = ['y', 'percentage']

# get the count of each tool_class
df_c = df.groupby('y').size().to_frame().reset_index()
df_c.columns = ['y', 'count']

# merge the two dataframes
df_pc = df_p.merge(df_c, on='y')[['y', 'count', 'percentage']]

# show the percentage with only two decimal places
df_pc['percentage'] = df_pc['percentage'].round(2)
df_pc

,y,count,percentage
0,0,5352,97.26
1,1,151,2.74


In [17]:
# get the percentage of each y label
dfc = df.groupby(['case_tool_54','y']).count().reset_index()[['case_tool_54','y','id']].rename(columns={'id':'n_cuts'})
dfc

,case_tool_54,y,n_cuts
0,1,0,100
1,2,0,160
2,2,1,15
3,3,0,21
4,4,0,219
5,5,0,211
6,6,0,145
7,7,0,226
8,8,0,321
9,9,0,209


In [18]:
dfc.y.sum()

11